#Generative AI-Powered Meeting Assistant

##meta-llama3 Implementation

In [1]:
!pip install transformers gradio datasets sentencepiece langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/6

In [2]:
#!pip install transformers gradio datasets sentencepiece langchain langchain-community

import torch
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from huggingface_hub import login
from typing import Any, List, Mapping, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens


# Ensure device compatibility
device = "cuda" if torch.cuda.is_available() else "cpu"

# Log in to Hugging Face Hub - Replace 'YOUR_HUGGINGFACE_TOKEN' with your actual token
login(token='')

# Hugging Face model setup for LLaMA
model_name = "meta-llama/Llama-2-7b-chat-hf"  # Replace with your desired LLaMA model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)

# Function to generate text using the LLaMA model
def generate_response(context):
    """
    Generate a response using the LLaMA model.
    """
    inputs = tokenizer.encode(context, return_tensors="pt").to(device)
    outputs = model.generate(inputs, max_length=800, temperature=0.7, top_p=0.9)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Prompt template setup
temp = """
# Summarize the following context into a few concise key points in real-time. Focus only on essential details or actionable insights.
# Context: {context}
"""

pt = PromptTemplate(
    input_variables=["context"],
    template=temp
)

# LangChain integration for prompt
class CustomLLM(LLM):  # Inherit from langchain.llms.base.LLM
    """
    Wrapper to integrate custom LLaMA model into LangChain.
    """
    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        """Call out to LLaMA model and return response."""
        response = generate_response(prompt)  # Use your existing generate_response function
        if stop is not None:
            # This is in charge of ensuring that the generated text does not contain the stop tokens.
            response = enforce_stop_tokens(response, stop)
        return response


llm = CustomLLM()  # Instantiate the custom LLM
prompt_to_llama = LLMChain(llm=llm, prompt=pt)

# Speech-to-text function
def transcript_audio(audio_file):
    """
    Transcribe audio file and generate summarized insights.
    """
    # Initialize the speech recognition pipeline
    asr_pipeline = pipeline(
        "automatic-speech-recognition",
        model="openai/whisper-tiny.en",
        chunk_length_s=30
    )
    # Transcribe the audio file
    transcript_txt = asr_pipeline(audio_file, batch_size=8)["text"]
    # Generate response using LLaMA
    result = prompt_to_llama.run(transcript_txt)
    return result

# Gradio interface
audio_input = gr.Audio(sources="upload", type="filepath")
output_text = gr.Textbox()

iface = gr.Interface(
    fn=transcript_audio,
    inputs=audio_input,
    outputs=output_text,
    title="Audio Transcription and Summarization App",
    description="Upload an audio file to transcribe and summarize using LLaMA."
)

# Launch the Gradio interface
iface.launch(share=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

<ipython-input-2-a9af84c1eaa5>:71: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  prompt_to_llama = LLMChain(llm=llm, prompt=pt)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://35f3bc76dab93975e7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


##OpenAI Implementation

In [3]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.59.6
    Uninstalling openai-1.59.6:
      Successfully uninstalled openai-1.59.6


In [4]:
import os
import gradio as gr
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from transformers import pipeline

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = " "  # Replace with your actual API key

# Initialize the OpenAI LLM - Corrected model specification
llm = ChatOpenAI(temperature=0.7, max_tokens=800, model_name="gpt-4o")

# Define the prompt template
temp = """
#Summarize the following context into a few concise key points in real-time. Focus only on essential details or actionable insights.
#Context: {context}
"""

pt = PromptTemplate(input_variables=["context"], template=temp)

# Create the LLM chain
prompt_to_gpt4 = LLMChain(llm=llm, prompt=pt)

# Speech-to-text function using Whisper
def transcript_audio(audio_file):
    try:
        pipe = pipeline("automatic-speech-recognition", model="openai/whisper-tiny.en", chunk_length_s=30)
        transcript_txt = pipe(audio_file, batch_size=8)["text"]

        # Ensure transcript_txt is a string
        if not isinstance(transcript_txt, str):
            transcript_txt = str(transcript_txt)

        result = prompt_to_gpt4.run(transcript_txt)  # Use prompt_to_gpt4
        return result
    except Exception as e:
        # Print the error message and traceback
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()
        return f"An error occurred: {e}"

# Gradio interface
audio_input = gr.Audio(sources="upload", type="filepath")
output_text = gr.Textbox()

iface = gr.Interface(
    fn=transcript_audio,
    inputs=audio_input,
    outputs=output_text,
    title="Audio Transcription App",
    description="Upload the audio file"
)

iface.launch()

<ipython-input-4-1a320d74d680>:12: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.7, max_tokens=800, model_name="gpt-4o")


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8126512373460ac886.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
